In [1]:
# Dependencies
import pandas as pd
import numpy as np
import tweepy
from datetime import datetime
import matplotlib.pyplot as plt 
import time
import json
import seaborn as sb
from matplotlib import style
style.use('ggplot')

#Configuration
from config import consumer_key, consumer_secret, access_token, access_token_secret

# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()


In [2]:
# Twitter API Keys
consumer_key = consumer_key
consumer_secret = consumer_secret
access_token = access_token
access_token_secret = access_token_secret

In [3]:
# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, parser=tweepy.parsers.JSONParser())

In [4]:
#Variables
target_term = "\"New Tweet Analysis: \""
namelist = []

# Get all tweets from home feed
public_tweets = api.user_timeline("@jmendiola84", count = 1000)

# Loop through all tweets 
for tweet in public_tweets:
    tweet_text = tweet["text"]
    split_tweet = tweet_text.split(" ")
    
    #Conditionals to find valid accounts already used for Sentiment Analysis
    if (split_tweet[0] == "New" and split_tweet[1] == "Tweet" and split_tweet[2] == "Analysis:"):
        namelist.append(split_tweet[3])
    
    #Conditionals to find invalid accounts already used for Sentiment Analysis
    if(split_tweet[0] == "Sorry."):
        namelist.append(split_tweet[4])
        
#List of Twitter accounts already requested for Sentiment Analysis (incuding invalid accounts)
print(namelist)

['@NBCNews', '@notavalidaccount', '@FoxNews', '@CNN', '@CNNEE', '@cnnbrk', '@realDonaldTrump']


In [5]:
#Function to find and process requests
def SearchRequest():
    #Variables
    target_term = "\"@jmendiola84 Analyze: @\""
    
    # Search for all tweets
    public_tweets = api.search(target_term, count=500, result_type="recent")

    # Loop through all tweets
    for tweet in public_tweets["statuses"]:

        # Get ID and Author of most recent tweet directed to me
        tweet_id = tweet["id"]
        tweet_author = tweet["user"]["screen_name"]
        # Tweet text is removed to maintain only requested account name
        tweet_text = str(tweet["text"])
        candidate = tweet_text.replace("@jmendiola84 Analyze: @", "@")
        # Print the account to analyze
        print(f"Account to Analyze = {candidate}")
        #Conditional to validate if account has been previously analyzed
        if (candidate in namelist):
            print("This account has been previously analyzed")
        else: 
            #The account to be analyzed is added to the list of accounts already analyzed
            namelist.append(candidate)
            print(namelist)
            # Target Account
            target_user = candidate

            # Counter
            counter = 1

            # Variables for holding sentiments
            sentiments = []

            # Variable for max_id
            oldest_tweet = None

            try:
                # Loop through 25 pages of tweets (total 500 tweets)
                for x in range(25):
                    # Get all tweets from home feed
                    public_tweets = api.user_timeline(target_user, max_id = oldest_tweet)

                    # Loop through all tweets 
                    for tweet in public_tweets:
     
                        # Run Vader Analysis on each tweet
                        results = analyzer.polarity_scores(tweet["text"])
                        compound = results["compound"]
                        pos = results["pos"]
                        neu = results["neu"]
                        neg = results["neg"]
                        tweets_ago = counter
        
                        # Get Tweet ID, subtract 1, and assign to oldest_tweet
                        oldest_tweet = tweet["id"] - 1
        
                        # Add sentiments for each tweet into a list
                        sentiments.append({"Date": tweet["created_at"], 
                                           "Compound": compound,
                                           "Positive": pos,
                                           "Negative": neg,
                                           "Neutral": neu,
                                           "Tweets Ago": counter})
        
                        # Add to counter 
                        counter += 1
                    
                # Convert sentiments to DataFrame
                sentiments_pd = pd.DataFrame.from_dict(sentiments)
                
                # Create plot
                x_vals = sentiments_pd["Tweets Ago"]
                y_vals = sentiments_pd["Compound"]
                plt.plot(x_vals, y_vals, marker="o", linewidth=0.5, alpha=0.8)

                # Incorporate the other graph properties
                now = datetime.now()
                now = now.strftime("%Y-%m-%d")
                plt.title(f"Sentiment Analysis of Tweets ({now}) for {target_user}", {'fontsize':10})
                plt.xlim([x_vals.max(),x_vals.min()])
                plt.ylabel("Tweet Polarity")
                plt.xlabel("Tweets Ago")
                
                #Plot saved in PNG file
                plt.savefig(f"SentimentAnalysis{target_user}.png")
                
                #Plot is cleaned for the next analysis
                plt.clf()

    
                # Plot is tweeted with a message including name of account analyzed and name of requester account
                api.update_with_media(f"SentimentAnalysis{target_user}.png",f"New Tweet Analysis: {target_user} (Thanks @{tweet_author}!!)")
            #Error handling for invalid accounts.
            except tweepy.TweepError as e:
                api.update_status(f"Sorry. It seems like {target_user} is not a valid Twitter account.")

In [6]:
while(True):
        SearchRequest()
        time.sleep(300)

TweepError: Failed to send request: HTTPSConnectionPool(host='api.twitter.com', port=443): Max retries exceeded with url: /1.1/search/tweets.json?q=%22%40jmendiola84+Analyze%3A+%40%22&count=500&result_type=recent (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000001BD70BA92B0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed',))